In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.metrics import roc_curve, auc

# Load data

In [2]:
emotion_labels = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}
targets = {val for key, val in emotion_labels.items()} 

In [3]:
df = pd.read_csv('../data/fer_data.csv')
df.head()

,usage,emotion,pixels
0,train,3,108 83 63 65 89 111 121 121 125 111 103 105 11...
1,train,3,137 142 159 162 158 134 113 167 174 177 164 12...
2,train,3,111 148 155 167 181 191 194 193 195 199 203 20...
3,train,3,151 156 121 100 80 116 151 169 183 185 192 190...
4,train,3,248 187 149 130 97 140 139 117 197 251 231 255...


In [4]:
df_sift = pd.read_csv('../data/sift_data.csv')
df_sift.head()

,usage,emotion,histogram
0,train,3,4 3 6 1 1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,train,3,2 8 4 4 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,train,3,2 9 9 5 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,train,3,2 3 3 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,train,3,6 14 7 2 1 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [5]:
def load_data(df, sift):
    image_data = []
    
    for index, row in df.iterrows():
        if sift == 1:
            field = row.histogram
        else:
            field = row.pixels
        image = np.fromstring(field, dtype=int, sep=' ')
        #scale = np.fromstring(row.scaled_pixels, dtype=int, sep=' ')
        #image = np.reshape(image, (48, 48))
        image_data.append(image)
        #image_scaled.append(scale)
        
    return image_data

# SVM

## original dataset

In [51]:
df_train = df[df['usage']=='train']
df_test = df[df['usage']=='test']
train_data = load_data(df_train, 0)
train_target = list(df_train.emotion)
test_data = load_data(df_test, 0)
test_target = list(df_test.emotion)

In [52]:
X_train = np.array(train_data)
y_train = np.array(train_target)
X_test = np.array(test_data)
y_test = np.array(test_target)

In [44]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(28262, 2304)
(28262,)
(7066, 2304)
(7066,)


In [45]:
svm = SVC(C=5, random_state=1, kernel='rbf', gamma=0.01)
svm.fit(X_train, y_train)
y_pred1 = svm.predict(X_test)
print('Test Accuracy: %.3f' % svm.score(X_test, y_test))
y_test1 = y_test

Test Accuracy: 0.316


## SIFT dataset

In [6]:
df_train = df_sift[df_sift['usage']=='train']
df_test = df_sift[df_sift['usage']=='test']
train_data = load_data(df_train, 1)
train_target = list(df_train.emotion)
test_data = load_data(df_test, 1)
test_target = list(df_test.emotion)

In [7]:
X_train = np.array(train_data)
y_train = np.array(train_target)
X_test = np.array(test_data)
y_test = np.array(test_target)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(28257, 29)
(28257,)
(7065, 29)
(7065,)


In [9]:
svm = SVC(C=5, random_state=1, kernel='rbf', gamma=0.01)
svm.fit(X_train, y_train)
y_pred2 = svm.predict(X_test)
print('Test Accuracy: %.3f' % svm.score(X_test, y_test))
y_test2 = y_test

Test Accuracy: 0.251


## Tuning hyperparameters 

## Visualizing learning curves and validation curves

# Evaluation

## Confusion matrix

In [10]:
y_pred = y_pred2
y_test = y_test2
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

## Parformance matrics

In [ ]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='macro'))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='macro'))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='macro'))
print('Accuracy: %.3f' % accuracy_score(y_true=y_test, y_pred=y_pred))

## Classification reports

In [ ]:
print(classification_report(y_test, y_pred, target_names=targets))

## ROC curve